In [20]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
import datetime

# Load the dataset
data = pd.read_csv('/content/sample_data/Play Store Data.csv')
data['Installs'] = (
    data['Installs']
    .astype(str)
    .str.replace(r'[+,]', '', regex=True)
    .replace('Free', '0')  # Handle 'Free' values
    .astype(float)
)
data['Reviews'] = pd.to_numeric(data['Reviews'], errors='coerce')

data['Type'] = data['Type'].fillna('Free')

def is_within_time_range(start_hour, end_hour):
    now = datetime.datetime.now().time()
    return start_hour <= now.hour < end_hour

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])
server = app.server

app.layout = html.Div([
    html.H1("Play Store Data Analysis Dashboard", style={'textAlign': 'center', 'color': 'cyan'}),
    html.Hr(),
    dcc.Tabs(id="tabs", value='tab1', children=[
        dcc.Tab(label='Revenue vs. Installs', value='tab1', style={'backgroundColor': '#333', 'color': 'white'}),
        dcc.Tab(label='Ratings & Reviews per Category', value='tab2', style={'backgroundColor': '#333', 'color': 'white'}),
        dcc.Tab(label='Installs by Category (Map)', value='tab3', style={'backgroundColor': '#333', 'color': 'white'}),
        dcc.Tab(label='Install Trends Over Time', value='tab4', style={'backgroundColor': '#333', 'color': 'white'}),
        dcc.Tab(label='Correlation Matrix', value='tab5', style={'backgroundColor': '#333', 'color': 'white'}),
    ], style={'backgroundColor': '#222'}),
    html.Div(id='tabs-content', style={'backgroundColor': '#111', 'padding': '20px', 'borderRadius': '10px'}),
    html.Button("Download Dashboard", id="download-button", style={'margin': '20px'}),
    dcc.Download(id="download-dashboard")
])

@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value')
)
def update_tab(tab_name):
    if tab_name == 'tab1':
        return dcc.Graph(figure=update_scatter_plot())
    elif tab_name == 'tab2' and is_within_time_range(15, 17):
        return dcc.Graph(figure=update_grouped_bar_chart())
    elif tab_name == 'tab3' and is_within_time_range(18, 20):
        return dcc.Graph(figure=update_choropleth_map())
    elif tab_name == 'tab4' and is_within_time_range(18, 21):
        return dcc.Graph(figure=update_time_series())
    elif tab_name == 'tab5' and is_within_time_range(14, 16):
        return dcc.Graph(figure=update_heatmap())
    else:
        return html.H3("This section is not available at this time.", style={'textAlign': 'center', 'color': 'red'})

@app.callback(
    Output("download-dashboard", "data"),
    Input("download-button", "n_clicks"),
    prevent_initial_call=True
)
def download_dashboard(n_clicks):
    with open("dashboard.html", "w") as file:
        file.write(app.index_string)
    return dcc.send_file("dashboard.html")

# Functions for analysis

def update_scatter_plot():
    paid_apps = data[data['Type'] == 'Paid'].copy()
    paid_apps['Revenue'] = paid_apps['Installs'] * paid_apps['Price'].str.replace('$', '', regex=False).astype(float)
    fig = px.scatter(paid_apps, x='Installs', y='Revenue', color='Category', log_x=True, log_y=True, title='Revenue vs. Installs for Paid Apps')
    fig.update_layout(template='plotly_dark')
    return fig

def update_grouped_bar_chart():
    filtered_data = data.groupby('Category').agg({'Rating': 'mean', 'Reviews': 'sum'}).reset_index()
    fig = go.Figure()
    fig.add_trace(go.Bar(x=filtered_data['Category'], y=filtered_data['Rating'], name='Average Rating', marker_color='cyan'))
    fig.add_trace(go.Bar(x=filtered_data['Category'], y=filtered_data['Reviews'], name='Total Reviews', yaxis='y2', marker_color='magenta'))
    fig.update_layout(title='Ratings & Reviews per Category', yaxis=dict(title='Average Rating'), yaxis2=dict(title='Total Reviews', overlaying='y', side='right', type='log'), barmode='group', template='plotly_dark')
    return fig

def update_choropleth_map():
    category_installs = data.groupby('Category', as_index=False)['Installs'].sum()
    top_categories = category_installs.nlargest(5, 'Installs')
    top_categories['Country'] = ['USA', 'India', 'Brazil', 'Russia', 'Germany']
    fig = px.choropleth(top_categories, locations="Country", locationmode="country names", color="Installs", hover_name="Category", title="Global Installs by Top App Categories")
    fig.update_layout(template='plotly_dark')
    return fig

def update_time_series():
    df = data.copy()
    df['Last Updated'] = pd.to_datetime(df['Last Updated'], errors='coerce')
    df['Year-Month'] = df['Last Updated'].dt.to_period('M')
    grouped_df = df.groupby(['Year-Month', 'Category'])['Installs'].sum().reset_index()
    grouped_df['Year-Month'] = grouped_df['Year-Month'].astype(str)
    grouped_df['Year-Month'] = pd.to_datetime(grouped_df['Year-Month'])
    fig = px.line(grouped_df, x='Year-Month', y='Installs', color='Category', title="Install Trends Over Time by Category")
    fig.update_layout(template='plotly_dark')
    return fig

def update_heatmap():
    numeric_data = data[['Installs', 'Rating', 'Reviews']].dropna()
    corr_matrix = numeric_data.corr()
    fig = px.imshow(corr_matrix, text_auto=True, color_continuous_scale='Viridis', title="Correlation Matrix for Filtered Data")
    fig.update_layout(template='plotly_dark')
    return fig

if __name__ == '__main__':
    app.run_server(debug=False)


<IPython.core.display.Javascript object>